In [1]:
from fusemix.simulation import Simulation
from fusemix.imputation import MultipleImputer
from gower import gower_matrix

import fusemix.clustering as clustering_utils

from sklearn.impute import KNNImputer

import warnings
import pickle
import importlib

import pandas as pd

from time import sleep 
from tqdm import tqdm_notebook


importlib.reload(clustering_utils)

warnings.filterwarnings("ignore")

In [2]:
def read_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    

def write_pickle(var, path):
    with open(path, 'wb') as f:
       pickle.dump(var, f)

In [3]:
complete_data = read_pickle("test_data/loaded_data.pkl")

In [4]:
complete_data[15].keys()

dict_keys(['X', 'y', 'cat_variables', 'cat_variables_in_X', 'cat_mask', 'id', 'X_complete', 'y_complete', 'num_classes'])

In [5]:


datasets = [14,15,17,33,45,915]

n_runs = 20

md_param_grid = {
    'props': [0.75,1.],
    'mf_proportions': [0.5,0.75],
    'mnar_proportions': [0.,0.25]
}

#external and internal metrics
results_fusemix = {}
results_singleimput = {}
results_singleimput_knn = {}
results_kpod = {}


metrics = ['ari','ami','vm','cs','sh','ch','db']
internal_metrics = ['sh','ch','db']
external_metrics = ['ari','ami','vm','cs']
external_metrics_known = []

for em in external_metrics:
    external_metrics_known.append("known_"+em)


result_dumps = [results_fusemix,results_kpod, results_singleimput,results_singleimput_knn]



In [6]:
# Initialize all results structures

for d in tqdm_notebook(datasets, desc="Datasets"):

    results_fusemix[d] = {}
    results_kpod[d] = {}
    results_singleimput[d] = {}
    results_singleimput_knn[d] = {}
    
    for prop in tqdm_notebook(md_param_grid['props'], desc="props"):
        for mf_proportion in tqdm_notebook(md_param_grid['mf_proportions'], desc="mf_proportions"):
            for mnar_proportion in tqdm_notebook(md_param_grid['mnar_proportions'], desc="mnar_proportions"):
                for dump_ in result_dumps:
                    dump_[d][(prop,mf_proportion,mnar_proportion)] = {}
                    for m_ in metrics:
                        dump_[d][(prop,mf_proportion,mnar_proportion)][m_] = []
                    for m_ in external_metrics_known:
                        dump_[d][(prop,mf_proportion,mnar_proportion)][m_] = []

Datasets:   0%|          | 0/6 [00:00<?, ?it/s]

props:   0%|          | 0/2 [00:00<?, ?it/s]

mf_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

...

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mf_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

## 1 - MICE-related algorithms

In [7]:


for d in tqdm_notebook(datasets, desc="Datasets"):

    gower_dist_complete = gower_matrix(complete_data[d]['X_complete'],cat_features=complete_data[d]['cat_mask'])

    for prop in tqdm_notebook(md_param_grid['props'], desc="props", leave=False):
        for mf_proportion in tqdm_notebook(md_param_grid['mf_proportions'], desc="mf_proportions", leave=False):
            for mnar_proportion in tqdm_notebook(md_param_grid['mnar_proportions'], desc="mnar_proportions", leave=False):       
                for seed in tqdm_notebook(range(n_runs), desc = "seeds", leave=False):
                    # read data
                    directory = "test_data/"+str(d)+"/"+str(prop)+"_"+str(mf_proportion)+"_"+str(mnar_proportion)
                    simulated_data = read_pickle(directory+"/pipeline_"+str(seed)+".pickle")

                    # MICE imputation
                    mice_imputer = MultipleImputer(incomplete_data=simulated_data.incomplete_data, 
                                                   seed = seed,
                                                   num_imputations=10,
                                                   mean_match_candidates=10,
                                                   mean_match_strategy = "Normal")
                    mice_imputer.run_mice(iterations=2, num_estimators = 100)
                    
                    imputed_data = mice_imputer.get_multiple_imputations()
                    single_imputation = imputed_data[0]
                    

                    fusemix_labels = clustering_utils.compute_fusemix(multiple_imputed_data=imputed_data,
                                                  cat_mask=complete_data[d]['cat_mask'],
                                                  num_clusters=complete_data[d]['num_classes'],
                                                  nn_snf=10,
                                                  seed=seed)

                    single_imput_labels = clustering_utils.compute_spectral(complete_data=single_imputation,
                                                                cat_mask=complete_data[d]['cat_mask'],
                                                                num_clusters=complete_data[d]['num_classes'],
                                                                seed=seed
                                                                )
                    

                    cca_labels = clustering_utils.compute_spectral(complete_data=complete_data[d]['X_complete'],
                                                                cat_mask=complete_data[d]['cat_mask'],
                                                                num_clusters=complete_data[d]['num_classes'],
                                                                seed=seed
                                                                )

                    # external metrics calculation and save

                    fusemix_res = clustering_utils.external_metrics(cca_labels, fusemix_labels)
                    single_imput_res = clustering_utils.external_metrics(cca_labels, single_imput_labels)

                    # external on known classes 
                    fusemix_known = clustering_utils.external_metrics(complete_data[d]['y_complete'].values.flatten(), fusemix_labels)
                    single_imput_known = clustering_utils.external_metrics(complete_data[d]['y_complete'].values.flatten(), single_imput_labels)

                    # internal metrics calculation and save
                    fusemix_internal = clustering_utils.internal_metrics(fusemix_labels, gower_dist_complete, complete_data[d]['X_complete'])
                    singleimput_internal = clustering_utils.internal_metrics(single_imput_labels, gower_dist_complete, complete_data[d]['X_complete'])
            
                    for m_ in external_metrics:
                        results_fusemix[d][(prop,mf_proportion,mnar_proportion)][m_].append(fusemix_res[m_])
                        results_singleimput[d][(prop,mf_proportion,mnar_proportion)][m_].append(single_imput_res[m_])

                    for m_ in external_metrics:
                        results_fusemix[d][(prop,mf_proportion,mnar_proportion)]["known_"+m_].append(fusemix_known[m_])
                        results_singleimput[d][(prop,mf_proportion,mnar_proportion)]["known_"+m_].append(single_imput_known[m_])

                    for m_ in internal_metrics:
                        results_fusemix[d][(prop,mf_proportion,mnar_proportion)][m_].append(fusemix_internal[m_])
                        results_singleimput[d][(prop,mf_proportion,mnar_proportion)][m_].append(singleimput_internal[m_])


                    sleep(0.01)
    # save for dataset
    write_pickle(results_fusemix[d],"output_simulation/"+str(d)+"fusemix.pickle")
    write_pickle(results_singleimput[d],"output_simulation/"+str(d)+"singleimput.pickle")





     

Datasets:   0%|          | 0/6 [00:00<?, ?it/s]

props:   0%|          | 0/2 [00:00<?, ?it/s]

mf_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

...

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

In [8]:
write_pickle(results_fusemix,"output_simulation/fusemix.pickle")
write_pickle(results_singleimput,"output_simulation/singleimput.pickle")

               

## 2 - Kpod and KNN 


In [9]:


for d in tqdm_notebook(datasets, desc="Datasets"):

    gower_dist_complete = gower_matrix(complete_data[d]['X_complete'],cat_features=complete_data[d]['cat_mask'])

    for prop in tqdm_notebook(md_param_grid['props'], desc="props", leave=False):
        for mf_proportion in tqdm_notebook(md_param_grid['mf_proportions'], desc="mf_proportions", leave=False):
            for mnar_proportion in tqdm_notebook(md_param_grid['mnar_proportions'], desc="mnar_proportions", leave=False):
                
                for seed in tqdm_notebook(range(n_runs), desc = "seeds", leave=False):
                    # read data
                    directory = "test_data/"+str(d)+"/"+str(prop)+"_"+str(mf_proportion)+"_"+str(mnar_proportion)
                    simulated_data = read_pickle(directory+"/pipeline_"+str(seed)+".pickle")

        
                    # KNN imputation
                    knn_imputer = KNNImputer(n_neighbors=10)
                    single_knn_imputation = knn_imputer.fit_transform(simulated_data.incomplete_data)


                    kpod_labels = clustering_utils.compute_kpod(simulated_data.incomplete_data,
                                            num_clusters=complete_data[d]['num_classes'])[0]

                    single_imput_knn_labels = clustering_utils.compute_spectral(complete_data=single_knn_imputation,
                                                                cat_mask=complete_data[d]['cat_mask'],
                                                                num_clusters=complete_data[d]['num_classes'],
                                                                seed=seed
                                                                )
                    cca_labels = clustering_utils.compute_spectral(complete_data=complete_data[d]['X_complete'],
                                                                cat_mask=complete_data[d]['cat_mask'],
                                                                num_clusters=complete_data[d]['num_classes'],
                                                                seed=seed
                                                                )

                    # external metrics calculation and save
                    single_imput_knn_res = clustering_utils.external_metrics(cca_labels, single_imput_knn_labels)
                    kpod_res = clustering_utils.external_metrics(cca_labels, kpod_labels)

                    # external on known classes 
                    single_imput_knn_known = clustering_utils.external_metrics(complete_data[d]['y_complete'].values.flatten(), single_imput_knn_labels)
                    kpod_known = clustering_utils.external_metrics(complete_data[d]['y_complete'].values.flatten(), kpod_labels)

                    # internal metrics calculation and save
                    singleimput_knn_internal = clustering_utils.internal_metrics(single_imput_knn_labels, gower_dist_complete, complete_data[d]['X_complete'])
                    kpod_internal = clustering_utils.internal_metrics(kpod_labels, gower_dist_complete, complete_data[d]['X_complete'])

                    
                    for m_ in external_metrics:
                        results_kpod[d][(prop,mf_proportion,mnar_proportion)][m_].append(kpod_res[m_])
                        results_singleimput_knn[d][(prop,mf_proportion,mnar_proportion)][m_].append(single_imput_knn_res[m_])

                    for m_ in external_metrics:
                        results_kpod[d][(prop,mf_proportion,mnar_proportion)]["known_"+m_].append(kpod_known[m_])
                        results_singleimput_knn[d][(prop,mf_proportion,mnar_proportion)]["known_"+m_].append(single_imput_knn_known[m_])

                    for m_ in internal_metrics:
                        results_kpod[d][(prop,mf_proportion,mnar_proportion)][m_].append(kpod_internal[m_])
                        results_singleimput_knn[d][(prop,mf_proportion,mnar_proportion)][m_].append(singleimput_knn_internal[m_])

                    sleep(0.01)
    # save for dataset
    write_pickle(results_singleimput_knn[d],"output_simulation/"+str(d)+"singleimput_knn.pickle")
    write_pickle(results_kpod[d],"output_simulation/"+str(d)+"kpod.pickle")


Datasets:   0%|          | 0/6 [00:00<?, ?it/s]

props:   0%|          | 0/2 [00:00<?, ?it/s]

mf_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

...

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

mnar_proportions:   0%|          | 0/2 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

seeds:   0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
write_pickle(results_singleimput_knn,"output_simulation/singleimput_knn.pickle")
write_pickle(results_kpod,"output_simulation/kpod.pickle")